In [ ]:
import pandas as pd
import numpy as np
import re 

import requests
from bs4 import BeautifulSoup

import time

#Сбор данных про банки, у которых была отозвана лицензия
class License_revoke_list: 
    def __init__(self):
        self.dfs_list = []
        self.final_df = pd.DataFrame()

    def page_getter(self):
        for p in range(5): #5
            page = p + 1
            url =f'https://www.banki.ru/banks/memory/?sort=date&direction=desc&PAGEN_1={page}'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            yield soup
            
    def table_getter(self):
        for page in self.page_getter():
            table = page.find_all('table', {'class' : 'standard-table standard-table--row-highlight margin-bottom-default'})[0]
            df = pd.read_html(str(table))[0]
            df = df[['банк', 'причина', 'дата отзыва']]
            df['дата отзыва'] = pd.to_datetime(df['дата отзыва'], format = "%d.%m.%Y")
            self.dfs_list.append(df)
            print('page_downloaded')

    def concatter(self):
        self.table_getter()
        for table in self.dfs_list:
            self.final_df = pd.concat([self.final_df, table])

cls = License_revoke_list()
cls.concatter()
license_revoke_df = cls.final_df

cls = License_revoke_list()
cls.concatter()
license_revoke_df = cls.final_df


#Собираем информацию о количестве подразделений каждого банка из ЦБ РФ

url = 'https://www.cbr.ru/banking_sector/credit/cstat/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table', {'class' : 'data levels'})[0]
df = pd.read_html(str(table))[0]
df.to_excel('num_offices.xlsx', index= False)

In [1]:
import pandas as pd
import numpy as np
import re 

import requests
from bs4 import BeautifulSoup

import time

### Собираем список банков у которых была отозвана лицензия

In [83]:
class License_revoke_list: 
    def __init__(self):
        self.dfs_list = []
        self.final_df = pd.DataFrame()

    def page_getter(self):
        for p in range(5): #5
            page = p + 1
            url =f'https://www.banki.ru/banks/memory/?sort=date&direction=desc&PAGEN_1={page}'
            response = requests.get(url)
            soup = BeautifulSoup(response.content, 'html.parser')
            yield soup
            
    def table_getter(self):
        for page in self.page_getter():
            table = page.find_all('table', {'class' : 'standard-table standard-table--row-highlight margin-bottom-default'})[0]
            df = pd.read_html(str(table))[0]
            df = df[['банк', 'причина', 'дата отзыва']]
            df['дата отзыва'] = pd.to_datetime(df['дата отзыва'], format = "%d.%m.%Y")
            self.dfs_list.append(df)
            print('page_downloaded')

    def concatter(self):
        self.table_getter()
        for table in self.dfs_list:
            self.final_df = pd.concat([self.final_df, table])

In [84]:
cls = License_revoke_list()
cls.concatter()
license_revoke_df = cls.final_df

page_downloaded
page_downloaded
page_downloaded
page_downloaded
page_downloaded


In [86]:
license_revoke_df.to_excel('license_revoke_df.xlsx', index=False)

### Информация о количестве подразделений из цб рф

In [ ]:
url = 'https://www.cbr.ru/banking_sector/credit/cstat/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table', {'class' : 'data levels'})[0]
df = pd.read_html(str(table))[0]
df.to_excel('num_offices.xlsx', index= False)

In [101]:
df = df.loc[1:]

In [118]:
def inter(x):
    try:
        return int(x)
    except:
        return None

df['Всего подразделений'] = df['Всего подразделений'].apply(lambda x: inter(x))

C:\Users\Masha\AppData\Local\Temp\ipykernel_38332\2265386854.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Всего подразделений'] = df['Всего подразделений'].apply(lambda x: inter(x))


In [ ]:
df1 = df.groupby('Наименование КО')[['Всего подразделений']].sum()
df1 = df1.reset_index()
df2 = df1.sort_values(by = 'Всего подразделений')[7:]
df2.to_excel('num_offices_2024.xlsx', index= False)

In [164]:
pattern = re.compile(r'Тинькофф') # color, colour - будет засчитано
# match = re.findall(pattern, s)

In [167]:
df2['Наименование КО'].apply(lambda x: re.findall(pattern, x))

219    []
192    []
194    []
93     []
195    []
       ..
316    []
37     []
95     []
181    []
337    []
Name: Наименование КО, Length: 356, dtype: object

In [186]:
df2[df2['Наименование КО'].str.contains('Юни')]

,Наименование КО,Всего подразделений
251,ООО «НКО „Вестерн Юнион ДП Восток“,1.0
163,АО ЮниКредит Банк,58.0


In [187]:
naming = pd.read_excel('full_dfs/bank_ids.xlsx')

In [188]:
naming

,bank_name,id
0,Сбербанк,322
1,ВТБ,327
2,Газпромбанк,2764
3,Альфа-Банк,325
4,НКЦ (Национальный клиринговый центр),191161
...,...,...
348,«ИНЭКО»,5011664
349,ВБРР,16995
350,ПСБ,5306
351,СМП Банк,17200


In [189]:
df2

,Наименование КО,Всего подразделений
219,НКО «МОНЕТА» (ООО),1.0
192,ИНГ БАНК (ЕВРАЗИЯ) АО,1.0
194,КБ «Гарант-Инвест» (АО),1.0
93,АО «РФК-банк»,1.0
195,КБ «Дж.П. Морган Банк Интернешнл» (ООО),1.0
...,...,...
316,ПАО «Промсвязьбанк»,851.0
37,АО «АЛЬФА-БАНК»,1259.0
95,АО «Россельхозбанк»,1394.0
181,Банк ВТБ (ПАО),1673.0
